<a href="https://colab.research.google.com/github/margaridagomes/dataeng-basic-course/blob/main/spark_streaming/examples/example_3_api_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up PySpark

In [ ]:
%pip install pyspark

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local').appName('Test streaming').getOrCreate()
sc = spark.sparkContext

In [2]:
!rm -rf /content/landing
!rm -rf /content/bronze
!mkdir -p /content/landing

## Simulate producer:
- extract data from API
- store data as json in the lake
- run task async

In [3]:
import requests
from pyspark.sql.types import *
import json
import datetime
import asyncio

async def ingest_from_api(url: str, table: str, schema: StructType = None):
  response = requests.get(url)
  timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
  if response.status_code == 200:
    data = response.json()
    with open(f"/content/landing/{table}_{int(timestamp)}.json", "w") as f:
        json.dump(data, f)

async def producer(loop: int, interval_time: int):
  for i in range(loop):
    await ingest_from_api("https://api.carrismetropolitana.pt/vehicles", "vehicles")
    await ingest_from_api("https://api.carrismetropolitana.pt/lines", "lines")
    await asyncio.sleep(interval_time)

async def main():
  asyncio.create_task(producer(10, 30))

await main()

- Read from /content/landing as streaming
- store data in memory (for testing)
- store data in the bronze layer

In [4]:
from pyspark.sql.types import *

vehicle_schema = StructType([StructField('bearing', IntegerType(), True),
                             StructField('block_id', StringType(), True),
                             StructField('current_status', StringType(), True),
                             StructField('id', StringType(), True),
                             StructField('lat', FloatType(), True),
                             StructField('line_id', StringType(), True),
                             StructField('lon', FloatType(), True),
                             StructField('pattern_id', StringType(), True),
                             StructField('route_id', StringType(), True),
                             StructField('schedule_relationship', StringType(), True),
                             StructField('shift_id', StringType(), True),
                             StructField('speed', FloatType(), True),
                             StructField('stop_id', StringType(), True),
                             StructField('timestamp', TimestampType(), True),
                             StructField('trip_id', StringType(), True)])

stream = spark.readStream.format("json").schema(vehicle_schema).load("/content/landing/vehicles*")

dedup = stream.dropDuplicates()

In [5]:
# using memory for testing
try:
  if query.isActive:
    query.stop()
except:
  pass

query = (dedup.writeStream.format("memory").option("queryName", "vehicles").start())

In [17]:
spark.sql("select * from vehicles").show()

+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+
|bearing|            block_id|current_status|      id|      lat|line_id|      lon|pattern_id|route_id|schedule_relationship|    shift_id|    speed|stop_id|          timestamp|             trip_id|
+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+
|      0|             3039-13|    STOPPED_AT| 42|2110| 38.81079|   2725|-9.118859|  2725_0_1|  2725_0|            SCHEDULED|        3169|      0.0| 070511|2025-07-11 21:50:17|2725_0_1|1|3|2230...|
|    316|           7_7205-13|   INCOMING_AT| 41|1153|38.784954|   1204|-9.321782|  1204_2_1|  1204_2|            SCHEDULED|        7254|5.8333335| 170169|2025-07-11 21:48:25|1204_2_1_2230_225...|
|      0|  VER_

In [18]:
!rm -rf /content/bronze

In [19]:
from pyspark.sql.functions import *

# watermark is necessary because of the aggregation
transformed = stream.withWatermark("timestamp", "60 seconds")
agg = (transformed
       .groupBy(window(transformed.timestamp, "5 minutes"), col("current_status"))
       .agg(min(transformed.timestamp).alias("init_timestamp"), count("*").alias("count")))

def insert_vehicles(df, batch_id):
  #df2 = df.groupBy("window").pivot("current_status").sum("count")
  df.write.format("parquet").mode("append").save("/content/bronze/vehicles")

# using memory for testing
query2 = (agg
          .writeStream
          .outputMode("append")
          .foreachBatch(insert_vehicles)
          .option("checkpointLocation", "/content/bronze/checkpoint")
          .trigger(processingTime='20 seconds')
          .start())

In [22]:
spark.read.format("parquet").load("/content/bronze/vehicles/*").show(100, False)

+------------------------------------------+--------------+-------------------+-----+
|window                                    |current_status|init_timestamp     |count|
+------------------------------------------+--------------+-------------------+-----+
|{2025-07-11 21:45:00, 2025-07-11 21:50:00}|IN_TRANSIT_TO |2025-07-11 21:48:40|183  |
|{2025-07-11 21:45:00, 2025-07-11 21:50:00}|INCOMING_AT   |2025-07-11 21:48:21|78   |
|{2025-07-11 21:45:00, 2025-07-11 21:50:00}|STOPPED_AT    |2025-07-11 21:48:23|86   |
+------------------------------------------+--------------+-------------------+-----+



## Report
- show vehicles by status in 5-min window time
- one line per window time

In [23]:
def pivot_data(df: DataFrame):
  result = df.orderBy("init_timestamp").groupBy("window").pivot("current_status").sum("count")
  result.show(100, False)

df = spark.read.format("parquet").load("/content/bronze/vehicles/*")
pivot_data(df)

+------------------------------------------+-----------+-------------+----------+
|window                                    |INCOMING_AT|IN_TRANSIT_TO|STOPPED_AT|
+------------------------------------------+-----------+-------------+----------+
|{2025-07-11 21:45:00, 2025-07-11 21:50:00}|78         |183          |86        |
+------------------------------------------+-----------+-------------+----------+



In [24]:
query2.stop()